In [40]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import grequests
import time
from datetime import date
import calendar
from config import key
import json

C:\Users\dsiddine\AppData\Local\Continuum\anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [41]:
music_file = "music_data.csv"
music_data_df = pd.read_csv(music_file)
music_data_df.head()

,Position,Track Name,Artist,Streams,URL,Date,Region
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,2017-01-01,ec
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,2017-01-01,ec
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,2017-01-01,ec
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,2017-01-01,ec
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,2017-01-01,ec


C:\Users\dsiddine\AppData\Local\Continuum\anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [42]:
music_data_df = music_data_df.rename(columns = {'Region':'country_codes'})
music_data_df['country_codes'] = music_data_df['country_codes'].str.upper()
music_data_df.head()

,Position,Track Name,Artist,Streams,URL,Date,country_codes
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,2017-01-01,EC
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,2017-01-01,EC
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,2017-01-01,EC
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,2017-01-01,EC
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,2017-01-01,EC


C:\Users\dsiddine\AppData\Local\Continuum\anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [63]:
artists = music_data_df[["Artist"]].groupby(["Artist"]).count().reset_index()
artists["Artist"]

0            #TocoParaVos
1             $uicideBoy$
2                  *NSYNC
3                     -M-
4               070 Shake
5                    100%
6                 101Barz
7                    10cm
8                   12:00
9                13 Block
10              13 Crowns
11                16yrold
12      187 Strassenbande
13                   1987
14                  1Kilo
15               2 Chainz
16              21 Savage
17              22 Savage
18                24Heavy
19                 257ers
20                   2NE1
21                   2Pac
22           3 Doors Down
23                    30Y
24                  3robi
25          4 Non Blondes
26             4Keus Gang
27       5 After Midnight
28                50 Cent
29                    53A
              ...        
6598                   阿達
6599                  陳以桐
6600                  陳嘉唯
6601                  陳念澤
6602                  陳思函
6603                  陳惠婷
6604                 陳李張王
6605        

C:\Users\dsiddine\AppData\Local\Continuum\anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [66]:
urls = []
for index,row in artists.iterrows(): 
    artist = row.Artist.replace("#","%23")
    url =  f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist}&api_key={key}&format=json"
    urls.append(url)


C:\Users\dsiddine\AppData\Local\Continuum\anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [67]:
# A list to hold our things to do via async
async_list = []

for u in urls:
    action_item = grequests.get(u, timeout=1000000)
    async_list.append(action_item)

response = grequests.map(async_list)


C:\Users\dsiddine\AppData\Local\Continuum\anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [68]:

response = [x for x in response if x is not None ]
len(response)

6340

C:\Users\dsiddine\AppData\Local\Continuum\anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [69]:
"""
for index,row in tracks_df_unique.head(1000).iterrows():
    try:  
        track = row.Track.replace("#","%23")
        artist = row.Artist.replace("#","%23")
        url =  f"http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={key}&artist={artist}&track={track}&format=json"
        response = requests.get(url).json()
        tracks_df_unique.at[index,"album"] = response["track"]["album"]["title"]
        tracks_df_unique.at[index,"genre"] = response["track"]["toptags"]["tag"][0]["name"]
        
    except (AttributeError,KeyError, IndexError):
        x=1

tracks_df_unique.head()
"""

'\nfor index,row in tracks_df_unique.head(1000).iterrows():\n    try:  \n        track = row.Track.replace("#","%23")\n        artist = row.Artist.replace("#","%23")\n        url =  f"http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={key}&artist={artist}&track={track}&format=json"\n        response = requests.get(url).json()\n        tracks_df_unique.at[index,"album"] = response["track"]["album"]["title"]\n        tracks_df_unique.at[index,"genre"] = response["track"]["toptags"]["tag"][0]["name"]\n        \n    except (AttributeError,KeyError, IndexError):\n        x=1\n\ntracks_df_unique.head()\n'

C:\Users\dsiddine\AppData\Local\Continuum\anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [70]:
additional_info = []
row={}

for x in response:
    row ={}
    try:
        row["artist"] = x.json()["artist"]["name"]
        row["genre"] = x.json()["artist"]["tags"]["tag"][0]["name"]
        
        additional_info.append(row)
    except (IndexError,KeyError):
        a=1
additional_info = pd.DataFrame(additional_info)

C:\Users\dsiddine\AppData\Local\Continuum\anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [71]:
additional_info

,artist,genre
0,#TocoParaVos,cumbia
1,$uicideboy$,cloud rap
2,*NSYNC,pop
3,-M-,french
4,070 Shake,hip hop
5,100,chillout
6,10cm,Korean
7,12:00,pop
8,13 Block,Hip-Hop
9,187 Strassenbande,underground hip-hop


C:\Users\dsiddine\AppData\Local\Continuum\anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
